In [1]:
import os
import sys
import ogr
from gdalconst import *
import osgeo.gdal as gdal
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create path to data directory, make sure it exists
data_dir = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir),"Data")
assert os.path.isdir

In [4]:
# Define paths to image file and shape files + make sure it exists
imgf = os.path.join(data_dir, "PDX_DEM_WGS84_UTM10N_Zm_INT16.tif")
assert os.path.isfile(imgf)

In [6]:
# What are some things that we need to be able to do to a raster

# 1. look up # of bands
# 2. look up rows/cols
# 3. look up projection
# 4. look up geotransform
# 5. look up data type
# 6. get BBOX of raster
# 7. read all the data into memory
# 8. read a chunk of data into memory
# 9. write array to disk  (skipping)
# 10. write part of an array to disk (skipping)


## Find Number of Bands

In [7]:
def get_bandcount(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    n_bands = ds.RasterCount
    return n_bands
    
get_bandcount(imgf)

1

## Find Rows/Columns

In [9]:
def get_rowcols(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    return rows, cols

get_rowcols(imgf)

(3951, 6161)

## Find Projection

In [10]:
def get_projection(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    proj = ds.GetProjection()
    return proj

get_projection(imgf)


'PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]'

## Find GeoTransform

In [11]:
def get_geotransform(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    gt = ds.GetGeoTransform()
    return gt

get_geotransform(imgf)

(490893.2197361471, 10.0, 0.0, 5054008.957242726, 0.0, -10.0)

## Find Datatype of array

In [12]:
def get_native_dtype(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    gdal_dtype = ds.GetRasterBand(1).DataType
    return gdal_dtype

get_native_dtype(imgf)

3

## Find BBOX of array

In [17]:
def get_bbox(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    gt = ds.GetGeoTransform()
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    ulx = gt[0]
    uly = gt[3]
    lrx = ulx + (cols * gt[1])
    lry = uly + (rows * gt[5])
    # Return in NWSWE Order
    return uly, ulx, lry, lrx

get_bbox(imgf)

(5054008.957242726, 490893.2197361471, 5014498.957242726, 552503.2197361471)

## 7. Read all the data into memory

In [20]:
def read_all(file_path):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    n_bands = ds.RasterCount
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    array = np.array([ds.GetRasterBand(i + 1).ReadAsArray(0, 0, cols, rows)
                                  for i in range(n_bands)])
    return array

read_all(imgf)

array([[[ 63,  62,  62, ..., -99, -99, -99],
        [ 62,  62,  61, ..., -99, -99, -99],
        [ 62,  61,  61, ..., -99, -99, -99],
        ...,
        [ 66,  66,  66, ..., 118, 119, 119],
        [ 66,  66,  67, ..., 117, 118, 118],
        [ 67,  67,  68, ..., 118, 117, 117]]], dtype=int16)

In [21]:
# What are some assumptions we made in this function?
# 1. All the data will fit into memory
# 2. User wants data as 3D array

# How Could we fix these?
# 1. Calculate array size in memory ahead of extracting 
#     and raise an error if expected memory is greater than available
# 2. Reformat the array to 2D if only a single band

In [ ]:
# How could we make the function more dynamic?
# 1. Allow user to specify "window" to extract
# 2. Allow user to specify datatype to return
# 3. Include Error Raise if available memory will be exceeded
# 4. Allow user to specify a list of bands to extract if multi-band

## 8. Read a chunk of data into memory

In [91]:
def numpy_dtype_to_gdal(nptype):
    """
    Convert numpy data type to GDAL data type

    Parameters
    ----------
    nptype
    Returns
    -------

    """

    if nptype == np.bool:
        return gdal.GDT_Byte
    elif nptype == np.uint8:
        return gdal.GDT_Byte
    elif nptype == np.uint16:
        return gdal.GDT_UInt16
    elif nptype == np.uint32:
        return gdal.GDT_UInt32
    elif nptype == np.int16:
        return gdal.GDT_Int16
    elif nptype == np.int32:
        return gdal.GDT_Int32
    elif nptype == np.int64:
        return gdal.GDT_Int32
    elif nptype == np.float32:
        return gdal.GDT_Float32
    elif nptype == np.float64:
        return gdal.GDT_Float64
    elif nptype == np.complex64:
        return gdal.GDT_CFloat32
    elif nptype == np.complex128:
        return gdal.GDT_CFloat64
    else:
        raise ValueError("invalid numpy datatype provided")

def read_subset(file_path, band_list=None, tp=None, x_off=0, y_off=0, x_win=None, y_win=None):
    assert os.path.isfile(file_path)
    ds = gdal.Open(file_path, GA_ReadOnly)
    n_bands = ds.RasterCount
    rows = ds.RasterYSize
    cols = ds.RasterXSize
    
    if ds is None:
        raise IOError("bad path")
    ### Check optional arguments to make sure they are all valid -- LETS RAISE SOME ERRORS ###
    
    # BAND_LIST needs to be a list of integers, even if it is only 1 band.
    # First check to see if it was supplied - if not, make band_list all the bands               
    if band_list is None:
        band_list = [i+1 for i in range(n_bands)]
        
    # now that we have solved the None case, make sure band_list is a list not an int/tuple/etc.
    if not isinstance(band_list, list):
        raise TypeError("band_list is not a list - even single bands need to go in a list")
    
    # ds.GetRasterBand() starts counting at 1 so lets make sure the list doesn't have a 0.
    if 0 in band_list:
        raise ValueError("band_list contains a 0, please start your list at 1...")
        
    
    # Type -- lets make a decision to force users to provdie numpy data types, and then we can change them to gdal
    if tp is not None:
        try:
            gdal_tp = numpy_dtype_to_gdal(tp)
        except:
            raise TypeError("user needs to provide a valid numpy datatype")
            
    else:
        gdal_tp = ds.GetRasterBand(1).DataType

    # Lets write down the rules for x_off/y_off
    # 1. Must be an Integer (handle w/ ValueError)
    # 2. Must not be negative (handle w/ ValueError)
    # 3. Must not be greater than rows/columns, respectively(handle w/ ValueError)
    if (x_off is None) or (y_off is None):
        raise ValueError("x_off and y_off cannot be None")
    
    if (x_off < 0) or (y_off < 0):
        raise ValueError("x_off and y_off cannot be negative")

    if (x_off >= cols) or (y_off >= rows):
        raise ValueError("x_off and y_off cannot be greaterthan/equal to rows/columns")
        
    # Lets write down the rules for x_window/y_window if not None
    # 1. x_window and y_window need to be greater than 0
    # 3. x_off + x_window and y_off + y_window need to be less than cols/rows (handle with ValueError)
    
    if x_win is None:
        x_win = cols - x_off
    if y_win is None:
        y_win = rows - y_off
    
    if (x_win <= 0) or (y_win <= 0):
        raise ValueError("x_window and y_window must be greater than 0")
    if ((x_off + x_win) > cols) or ((y_off + y_win) > rows):
        raise ValueError(" x_off + x_window and y_off + y_window need to be less than cols/rows")
                

    array = np.array([ds.GetRasterBand(i).ReadAsArray(x_off, y_off,
                                                      x_win, y_win,
                                                      buf_type=gdal_tp) for i in band_list])
    if array.shape[0] == 1:
        return array[0]
    else:
        return array

data = read_subset(imgf, tp=np.float32)

## Okay this is cool, but now what?

In [92]:
# Well lets make some plots